In [51]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import re, json
from bs4 import BeautifulSoup
import urllib.request
from urllib.error import HTTPError
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm

In [52]:
tqdm.pandas()

In [58]:
train_data = pd.read_table('ratings_train.txt')
test_data = pd.read_table('ratings_test.txt')

In [61]:
train_data[:5] # 상위 5개 출력

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 솔직히 재미는 없다평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화스파이더맨에서 늙어보이기만 했던 커스틴 던...,1


In [55]:
test_data[:5]

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


In [60]:
# train_data 전처리

# 휴리스틱 검출규칙 : 비표준어에 대해 직관적인 방법으로 규칙을 만들어 교체
heuristic_rule = [
    ['ㅋㅋ+', ' ㅋㅋ '],
    ['ㄹㄹ+', ' ㄹㄹ '],
    ['ㅜㅜ+', ' ㅜㅜ '],
    ['ㅠㅠ+', ' ㅠㅠ '],
    ['ㄷㄷ+', ' ㄷㄷ '],
    ['ㅎㅎ+', ' ㅎㅎ '],
    ['ㅡㅡ+', ' ㅡㅡ '],
    ['^ +', ''],    
    ['  +', ' ']
]

train_data['document'] = train_data['document'].str.replace("[^0-9ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 한글숫자공백외 문자제거

for rule in tqdm(heuristic_rule):
    train_data['document'] = train_data['document'].str.replace(rule[0],rule[1]) # 휴리스틱 규칙적용

train_data['document'].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경

train_data.drop_duplicates(subset = ['document'], inplace=True) # document 열에서 중복인 내용이 있다면 중복 제거

train_data = train_data.dropna(how='any') # Null 값 제거

print('전처리 후 테스트용 샘플의 개수 :',len(train_data))

100%|██████████| 9/9 [00:01<00:00,  6.57it/s]


전처리 후 테스트용 샘플의 개수 : 143688


In [62]:
# test_data 전처리

test_data['document'] = test_data['document'].str.replace("[^0-9ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 한글숫자공백외 문자제거

for rule in tqdm(heuristic_rule):
    test_data['document'] = test_data['document'].str.replace(rule[0],rule[1]) # 휴리스틱 규칙적용

test_data['document'].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경

test_data.drop_duplicates(subset = ['document'], inplace=True) # document 열에서 중복인 내용이 있다면 중복 제거

test_data = test_data.dropna(how='any') # Null 값 제거

print('전처리 후 테스트용 샘플의 개수 :',len(test_data))

100%|██████████| 9/9 [00:00<00:00, 21.47it/s]

전처리 후 테스트용 샘플의 개수 : 48443


In [63]:
# 맞춤법 검사
def correct_sentence(s):
    # API 파라메터 설정
    params = urllib.parse.urlencode({ 
        "_callback": "", 
        "q": s 
    })

    try:
      # 네이버 맞춤법 검사 API 호출
      data = urllib.request.urlopen("https://m.search.naver.com/p/csearch/ocontent/spellchecker.nhn?" + params) #책의 url을 바꿈
      data = data.read().decode("utf-8")[1:-2]
      data = json.loads(data)
      data = data["message"]["result"]["html"]
      data = soup = BeautifulSoup(data, "html.parser").getText()
      # 리턴
      return data
    except HTTPError as e: # 호출 에러 시에는 입력값 그대로 반환
      return s

In [64]:
# 진행상황 체크해가면서 train_data 맞춤법 수정
train_data['correction'] = train_data['document'].progress_apply(correct_sentence)

100%|██████████| 143688/143688 [2:17:31<00:00, 17.41it/s]  


In [65]:
# 진행상황 체크해가면서 test_data 맞춤법 수정
test_data['correction'] = test_data['document'].progress_apply(correct_sentence)

100%|██████████| 48443/48443 [45:53<00:00, 17.59it/s]  


In [66]:
# 전처리 결과저장 ( 이후 훈련 시에 해당파일 사용 )
train_data.to_csv('ratings_train_preprocess.csv',sep='|')
test_data.to_csv('ratings_test_preprocess.csv',sep='|')